In [266]:
import pandas as pd
import numpy as np
import math
import sklearn

root = "synthetic.csv"

df_synthetic = pd.read_csv(root)

train_df = df_synthetic.sample(frac=0.80, random_state=42)
train_index = train_df.index
test_df = df_synthetic.drop(index=train_index)

Partie 1
1. Combien d’attributs comportent ces donn´ees ?
2. En combien de classes diff´erentes les instances sont-
elles cat´egoris´ees ?
3. Combien d’instances chaque classe compte-elle ?

In [267]:



print(train_df.head())
print("--------------------------")
print("--------------------------")
print(train_df.info())
print("--------------------------")
print("--------------------------")
nombre_classes = train_df['Class'].nunique()
print("Nombre de classes différentes :", nombre_classes)
print("--------------------------")
print("--------------------------")
valeurs_uniques_classes = train_df['Class'].unique()
print(valeurs_uniques_classes)
print("--------------------------")
print("--------------------------")
nombre_instances_par_classe = train_df['Class'].value_counts()
print(nombre_instances_par_classe)
print("--------------------------")
print("--------------------------")

         Attr_A     Attr_B       Attr_C      Attr_D      Attr_E       Attr_F  \
1085   8.248185  10.344703  1147.000044  101.365512  111.223836   822.249901   
2296  11.581467  12.024253  1247.894013   95.732086   85.265126  1012.536672   
208    9.052379  13.253354   875.890614   96.059527  109.192656  1023.766549   
1178  10.307719  11.627932  1161.791088   94.506795   75.607353  1058.761596   
282   13.668978   7.957635  1036.209573   99.972644  125.767378   767.796145   

         Attr_G     Attr_H     Attr_I      Attr_J       Attr_K     Attr_L  \
1085  10.123331  12.126327   6.531983   93.255796  1147.319151  10.613766   
2296   9.972115   9.001799  11.909924  128.966934   927.437658   8.906923   
208   11.044292  11.676797  11.670033   77.638248  1025.799580  11.602964   
1178   7.778766  10.110696   7.536797  100.426924  1092.576245   9.935735   
282   10.396706  13.765871   5.436818   85.150632   960.540053   9.806342   

           Attr_M      Attr_N  Class  
1085   986.337340

Il y a 14 attributs différents, 4 classes différentes: 0,1,2,3.
il y a:
908 Class 1,
674 Class 0,
472 Class 2,
244 Class 3.

In [268]:

# Séparation des attributs/features et des étiquettes/classes pour l'ensemble d'entraînement et de test
X_train = train_df.drop(columns=['Class'])
y_train = train_df['Class']
X_test = test_df.drop(columns=['Class'])
y_test = test_df['Class']


Fonction calcul de l'entropie

In [269]:
from math import log

def entropie(dataframe, attribut_cible='Class'):
    nb_lignes = dataframe.shape[0]
    occ = dataframe[attribut_cible].value_counts()
    h = 0
    for i in occ:
        p = i/nb_lignes
        h += p * log(p, 2)
    return -h

print("voici l'entropie:",entropie(df_synthetic))
print("voici l'entropie de l'attribut A:",entropie(df_synthetic, 'Attr_A'))

voici l'entropie: 1.8608867211835995
voici l'entropie de l'attribut A: 11.166163082646667


Méthode des quantiles

In [270]:
def gain(df, attribut='Attr_A', attribut_cible='Class'):
    entropie_local = entropie(df)
    gain_max = 0
    split_value = 0
    meilleures_partitions = [None, None]
    
    quartiles = df[attribut].quantile([0.25, 0.5, 0.75])
    
    for quartile_value in quartiles:
        tmp = 0  # On réinitialise tmp à zéro à chaque itération de la boucle externe
        
        tmp_partitions = [None, None]
        tmp_partitions[0] = df[df[attribut] <= quartile_value]
        tmp_partitions[1] = df[df[attribut] > quartile_value]
        
        for partition in tmp_partitions:
            tmp += (len(partition) / len(df)) * entropie(partition)
        
        tmp_gain = entropie_local - tmp
        
        if tmp_gain > gain_max:
            gain_max = tmp_gain
            split_value = quartile_value
            meilleures_partitions = tmp_partitions
            
    return attribut, gain_max, split_value, meilleures_partitions


In [271]:
la_liste_attributs = ['Attr_A', 'Attr_B', 'Attr_C', 'Attr_D', 'Attr_E', 'Attr_F', 'Attr_G', 'Attr_H', 'Attr_I', 'Attr_J', 'Attr_K', 'Attr_L', 'Attr_M', 'Attr_N']


Calcul du gain d'une partition et la détermination du meilleur partionnement. Pour l'entropie et le gain j'ai choisi le meilleur partionnement

In [272]:
def meilleur_attribut(dataframe, lesattributs):
    meilleur_gain = 0
    meilleur_attribut = None
    meilleur_quantile = None
    meilleures_partitions = None
    
    # Parcourir chaque attribut spécifié
    for attribut in lesattributs:
        # Calculer le gain d'information pour chaque attribut
        attribut_gain = gain(dataframe, attribut)
        
        # Mettre à jour le meilleur attribut si le gain d'information est maximal
        if attribut_gain[1] > meilleur_gain:
            meilleur_gain = attribut_gain[1]
            meilleur_attribut = attribut_gain[0]
            meilleur_quantile = attribut_gain[2]
            meilleures_partitions = attribut_gain[3]
    
    return meilleur_attribut, meilleur_gain, meilleur_quantile, meilleures_partitions

# Juste pour vérifier le meilleur attribut
meilleur_attribut_result = meilleur_attribut(df_synthetic, la_liste_attributs)
print("Meilleur attribut discriminant :", meilleur_attribut_result[0])
print("Gain d'information maximal :", meilleur_attribut_result[1])
print("Meilleur quantile :", meilleur_attribut_result[2])
print("Meilleures partitions :", meilleur_attribut_result[3])


Meilleur attribut discriminant : Attr_B
Gain d'information maximal : 0.16161530810503888
Meilleur quantile : 10.21752558689527
Meilleures partitions : [         Attr_A     Attr_B       Attr_C      Attr_D      Attr_E       Attr_F  \
1      8.264345   8.854181  1389.686814   99.526529   65.106526  1186.523399   
3      9.207661   9.346913   965.468523   89.176009  102.628284   743.913507   
7     11.971482   8.300276  1486.373115  102.629861  123.344768   859.180699   
14    12.447900   9.811186  1110.983429   94.227442  107.111364  1229.822227   
16     9.616927  10.060632  1304.177508   92.634518   94.093556  1241.461058   
...         ...        ...          ...         ...         ...          ...   
2287   8.009433   9.462369   802.498985  107.752730   93.582026  1163.643611   
2288  10.121841   9.013273  1019.564521  100.698598   90.176739  1133.595130   
2289  13.339844   6.189921  1372.348837   89.475740  107.824189   725.126646   
2292  10.882387   8.831101   979.510791  107.065

Maintenant on construit l'arbre

In [273]:
class Noeud:
    def __init__(self,attribut=None,split_value=None, prediction=None, feuille=False, gauche=None, droite=None):
        self.attribut = attribut
        self.split_value = split_value
        self.prediction = prediction
        self.feuille = feuille
        self.gauche = gauche
        self.droite = droite
        
    def __str__(self):
        return "<"+str(self.split_value)+" "+str(self.gauche)+" "+str(self.droite)+">"
    
    #fonction pour voir si c'est une feuille
    def isLeaf(self):
        return self.gauche is None and self.droite is None
    
    def __repr__(self):
        return str(self.split_value)
    
    def node_result(self, spacing=' '):
        result = ''
        for v in range(len(self.prediction.values)):
            result += ' Class ' + str(self.prediction.index[v]) + ' Count: ' + str(self.prediction.values[v]) + '\n' + spacing
        return result


In [274]:
def construction_arbre(dataframe, classe, liste_attributs, split, profondeur, profondeur_max):
    attribut, gain_ca, split, partitions = meilleur_attribut(dataframe, liste_attributs)
    prediction = dataframe[classe].value_counts()

    if ((profondeur >= profondeur_max) or (len(liste_attributs) == 0) or (gain_ca == 0)):
        return Noeud(attribut=attribut, split_value=split, prediction=prediction, feuille=True)

    liste_attributs.remove(attribut) #enllever l'attribut en haut

    branche_gauche = construction_arbre(partitions[0], classe, liste_attributs, split, profondeur + 1, profondeur_max)
    branche_droite = construction_arbre(partitions[1], classe, liste_attributs, split, profondeur + 1, profondeur_max)

    return Noeud(attribut=attribut, split_value=split, prediction=prediction, gauche=branche_gauche, droite=branche_droite)


In [275]:
def print_tree(noeud, spacing=''):
    if noeud is None:
        return
    if noeud.isLeaf():
        print(spacing + noeud.node_result(spacing))
        return

    print('{}[ Attribute: {} Split value: {}]'.format(spacing, noeud.attribut, noeud.split_value))

    print(spacing + '> True ')
    print_tree(noeud.gauche, spacing + '-')

    print(spacing + '> False')
    print_tree(noeud.droite, spacing + '-')


In [276]:
# Appel de la fonction construction_arbre pour créer l'arbre de décision avec une profondeur maximale de 8
arbre_decision = construction_arbre(train_df, 'Class', ['Attr_A', 'Attr_B', 'Attr_C', 'Attr_D', 'Attr_E', 'Attr_F', 'Attr_G', 'Attr_H', 'Attr_I', 'Attr_J', 'Attr_K', 'Attr_L', 'Attr_M', 'Attr_N'], 0, 0, 8)

# Affichage de l'arbre de décision
print_tree(arbre_decision)


[ Attribute: Attr_B Split value: 10.232649934230249]
> True 
-[ Attribute: Attr_I Split value: 9.515650018679326]
-> True 
--[ Attribute: Attr_C Split value: 877.205005150759]
--> True 
---[ Attribute: Attr_F Split value: 955.3216234090576]
---> True 
----[ Attribute: Attr_N Split value: 99.1351095872464]
----> True 
-----[ Attribute: Attr_D Split value: 97.62697748980612]
-----> True 
------[ Attribute: Attr_A Split value: 8.361927170403433]
------> True 
------- Class 0 Count: 4
-------
------> False
-------[ Attribute: Attr_E Split value: 135.0302150833443]
-------> True 
-------- Class 2 Count: 1
--------
-------> False
-------- Class 0 Count: 1
--------
-----> False
------ Class 0 Count: 16
------
----> False
-----[ Attribute: Attr_M Split value: 1022.4427450334299]
-----> True 
------[ Attribute: Attr_J Split value: 103.83081577874906]
------> True 
------- Class 2 Count: 4
-------
------> False
------- Class 1 Count: 1
-------
-----> False
------[ Attribute: Attr_G Split value: 

In [277]:
def inference(noeud, instance):
    # Si le noeud est une feuille, renvoyer la prédiction associée
    if noeud.feuille:
        return noeud.prediction.idxmax()
    
    # Sinon on descend dans l'arbre en fonction de la valeur de la caractéristique de l'instance
    if instance[noeud.attribut] <= noeud.split_value:
        return inference(noeud.gauche, instance)
    else:
        return inference(noeud.droite, instance)


In [278]:
def evaluate_tree_precision(arbre, X_test, y_test):
    correct_predictions = 0
    total_predictions = len(X_test)
    
    for idx, row in X_test.iterrows():
        prediction = inference(arbre, row)
        if prediction == y_test.loc[idx]:
            correct_predictions += 1
    
    accuracy = correct_predictions / total_predictions
    return accuracy


In [279]:

resultats_arbres = {}  #on stocke les performances ici

for profondeur_max in range(3, 9):  # Utilisation de range() pour itérer sur les valeurs de 3 à 8 inclus
    la_liste_attributs_loc = ['Attr_A', 'Attr_B', 'Attr_C', 'Attr_D', 'Attr_E', 'Attr_F', 'Attr_G', 'Attr_H', 'Attr_I', 'Attr_J', 'Attr_K', 'Attr_L', 'Attr_M', 'Attr_N']

    # Entraînement de l'arbre
    arbre = construction_arbre(train_df, 'Class', la_liste_attributs_loc, 0, 3, profondeur_max)
    
    # Évaluation de l'arbre sur l'ensemble de test
    accuracy = evaluate_tree_precision(arbre, X_test, y_test)  # À remplacer par la fonction d'évaluation réelle
    
    # Stockage de la précision de l'arbre et de sa profondeur dans le dictionnaire
    resultats_arbres[profondeur_max] = accuracy
    

# Affichage des résultats
for profondeur, accuracy in resultats_arbres.items():
    print(f"Profondeur maximale: {profondeur}, Précision: {accuracy}")


Profondeur maximale: 3, Précision: 0.40869565217391307
Profondeur maximale: 4, Précision: 0.48695652173913045
Profondeur maximale: 5, Précision: 0.5565217391304348
Profondeur maximale: 6, Précision: 0.6347826086956522
Profondeur maximale: 7, Précision: 0.6173913043478261
Profondeur maximale: 8, Précision: 0.5260869565217391


mes deux meilleures modèles ont l'air d'être ceux avec une profondeur maximale de 6 et 7. Cela reste à vérifier